## Importação das Bibiliotecas

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import pyodbc

## Criando uma SparkSession

In [2]:
spark = SparkSession.builder.appName("LerDadosEnem").getOrCreate()


## Processo de Leitura do CSV de Participantes


In [6]:
arqEnem = 'E:\Estudos\SQL\Datasets\microdados_enem_2022\DADOS\MICRODADOS_ENEM_2022.csv'

dfDadosGerais = spark.read.csv(arqEnem, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')


In [9]:
dfDadosSocioEconomico = dfDadosGerais.select(dfDadosGerais.columns[51:76])

In [10]:
dfDadosSocioEconomico.show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|Q001|Q002|Q003|Q004|Q005|Q006|Q007|Q008|Q009|Q010|Q011|Q012|Q013|Q014|Q015|Q016|Q017|Q018|Q019|Q020|Q021|Q022|Q023|Q024|Q025|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|   A|   A|   A|   B|   3|   B|   B|   B|   B|   B|   B|   B|   A|   B|   B|   B|   A|   A|   A|   A|   A|   A|   A|   A|   A|
|   D|   D|   E|   E|   3|   Q|   D|   E|   E|   E|   E|   E|   E|   E|   E|   E|   E|   B|   E|   B|   B|   E|   B|   E|   B|
|   E|   F|   A|   D|   2|   B|   A|   B|   C|   A|   A|   B|   B|   A|   A|   A|   A|   A|   A|   A|   A|   C|   A|   A|   B|
|   C|   A|   A|   B|   3|   A|   A|   C|   D|   A|   A|   B|   A|   B|   A|   B|   A|   A|   C|   A|   A|   C|   B|   B|   B|
|   D|   B|   A|   B|   2|   B|   A|   B|   B|   A|   A|   B|   A|   A|   A|   A|   A|   A|   A|   A|   A|   B|

In [11]:
dfPreencher = dfDadosSocioEconomico.fillna(-2)  # Preenche com -2(sem ref), por exemplo


# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [12]:
server_name = "jdbc:sqlserver://NOTEBOOK-LEO:1433"
database_name = "Enem"
table_name = "SocioEconomico"
username = "bi_leonardobo"
password = "0000"
url = server_name + ";" + "databaseName=" + database_name + ";"
print(url)


jdbc:sqlserver://NOTEBOOK-LEO:1433;databaseName=Enem;


In [13]:
# Nome da nova tabela que será criada
nomeTabela = table_name

In [14]:
# Configurações do banco de dados em ODBC para dropar
server_name = "NOTEBOOK-LEO\\SQLEXPRESS"
 
# Conecte-se ao banco de dados usando pyodbc
conn = pyodbc.connect(f"Driver={{SQL Server}};Server={server_name};Database={database_name};UID={username};PWD={password}")

# Crie um cursor
cursor = conn.cursor()

# Execute a operação DROP TABLE na tabela 'Participantes'
truncate_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}"
cursor.execute(truncate_query)

# Commit as alterações no banco de dados
conn.commit()

In [15]:
# Escreva os novos dados em 'Participantes' usando a conexão JDBC
dfPreencher.write \
        .format("jdbc") \
        .option("url", url) \
        .option("driver","com.microsoft.sqlserver.jdbc.SQLServerDriver")\
        .option("ssl", "true")\
        .option("trustServerCertificate", "true")\
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("SaveMode", "overWrite").save()
  


In [16]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')


Tabela SocioEconomico criada com sucesso no banco de dados.
